In [ ]:
import numpy as np
import pandas as pd
from binancepy import MarketData, Client

###TRADING RULES
QUANTPRE = {  'BTCUSDT': 3, 'ETHUSDT': 3, 'BCHUSDT': 2, 'XRPUSDT': 1, 'EOSUSDT': 1, 'LTCUSDT': 3, \
                'TRXUSDT': 0, 'ETCUSDT': 2, 'LINKUSDT': 2, 'XLMUSDT': 0, 'ADAUSDT': 0, 'XMRUSDT': 3, \
                'DASHUSDT': 3, 'ZECUSDT': 3, 'XTZUSDT': 1, 'BNBUSDT': 2, 'ATOMUSDT': 2, 'ONTUSDT': 1, \
                'IOTAUSDT': 1, 'BATUSDT': 1, 'VETUSDT': 0, 'NEOUSDT': 2, 'QTUMUSDT': 1, 'IOSTUSDT': 0 }
PRICEPRE = {  'BTCUSDT': 2, 'ETHUSDT': 2, 'BCHUSDT': 2, 'XRPUSDT': 4, 'EOSUSDT': 3, 'LTCUSDT': 2, \
              'TRXUSDT': 5, 'ETCUSDT':3, 'LINKUSDT': 3  , 'XLMUSDT': 5, 'ADAUSDT': 5, 'XMRUSDT': 2, \
              'DASHUSDT': 2, 'ZECUSDT': 2, 'XTZUSDT': 3, 'BNBUSDT': 3, 'ATOMUSDT': 3, 'ONTUSDT': 4, \
              'IOTAUSDT': 4, 'BATUSDT': 4, 'VETUSDT': 6, 'NEOUSDT': 3, 'QTUMUSDT': 3, 'IOSTUSDT': 6 }

SIDE = {'BUY': 1.0, 'SELL': -1.0}

testnet = True
if testnet:
    # Testnet
    apikey = '' ### INSERT your api key ###
    scrkey = '' ### INSERT your api secret ###
else:
    # Binance
    apikey = ''
    scrkey = ''

# Problem 1 : Safe Limit Price

In [ ]:
def get_possible_price(mk_data, side):
    '''
    Return a safe limit price available on the market
    '''
    mk_depth = mk_data.order_book(limit=5)
    bids = list(float(x[0]) for x in mk_depth['bids'])
    asks = list(float(x[0]) for x in mk_depth['asks'])
    try:
        lim = (side=='BUY')*(bids[0]+bids[1])/2 + (side=='SELL')*(asks[0]+asks[1])/2
        lim = round(lim, PRICEPRE[mk_data.symbol.upper()])
    except:
        print('Not enough bids and asks on the market!')
        lim = None
    return bids, asks, lim

In [ ]:
### TEST get_possible_price() function
symbol = 'BTCUSDT'
data = MarketData(testnet=testnet, symbol=symbol)
side = 'SELL'
bids, asks, price = get_possible_price(data, side)
if price is not None:
    print('side = ', side)
    if side=='BUY':
        print('best bid = {} \n2nd best bid = {} \nlimit price = {}'.format(bids[0], bids[1], price))
    elif side=='SELL':
        print('best ask = {} \n2nd best ask = {} \nlimit price = {}'.format(asks[0], asks[1], price))

# Problem 2 : Portfolio class

In [ ]:
### Portfolio Class:
class Portfolio:
    def __init__( self,
                  client,
                  tradeIns = []):
        '''
        Portfolio class
        '''
        self.client = client
        self.tradeIns = tradeIns.copy()
        self.orderSize = 0
        self.equityDist = {'BUY': 0, 'SELL': 0}
        self.locks = { 'BUY': [], 'SELL': []}
        
    def equity_distribution(self, longPct=0.5, shortPct=0.5, currency='USDT', orderPct=0.1):
        '''
        Retrun number of buy/sell orders with currenty equity
        
            longPct : percentage of equity assigned for buying
        
            shortPct : percentage of equity assigned for selling
        
            orderPct : percentage of equity for a single order
        '''
        balance = self.client.balance()
        equity, available = 0, 0
        for b in balance:
            if b['asset']==currency:
                equity, available = float(b['balance']), float(b['withdrawAvailable'])
                break
        long_equity = longPct*equity
        short_equity = shortPct*equity
        
        info = pd.DataFrame(self.client.position_info())
        if info.shape[0] > 0: 
            short_info = info[info['positionAmt'].astype(float) < 0]
            long_info = info[info['positionAmt'].astype(float) > 0]
            short_position = abs(short_info['positionAmt'].astype(float) @ short_info['entryPrice'].astype(float))
            long_position = abs(long_info['positionAmt'].astype(float) @ long_info['entryPrice'].astype(float))
        else:
            short_position = 0
            long_position = 0
        
        self.orderSize = round(orderPct*equity, 2)
        long_order = int((long_equity - long_position)/self.orderSize)
        short_order = int((short_equity - short_position)/self.orderSize)
        self.equityDist = {'BUY': long_order, 'SELL': short_order}
        return self.orderSize, long_order, short_order
        
    def position_locks(self, prelocks={ 'BUY': [], 'SELL': []}):
        '''
        Check for open positions and return a tradable instruments
        '''
        info = self.client.position_info()
        self.locks = prelocks
        for pos in info:
            amt = float(pos['positionAmt'])
            if amt < 0 and not pos['symbol'] in self.locks['SELL']: self.locks['SELL'].append(pos['symbol'])
            elif amt > 0 and not pos['symbol'] in self.locks['BUY']: self.locks['BUY'].append(pos['symbol'])
        drop_out = set(self.locks['SELL']).intersection(self.locks['BUY'])
        for s in drop_out: self.tradeIns.remove(s)
        return self.tradeIns

In [ ]:
### TEST Portfolio class
insIds = [ 'BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'LINKUSDT', 'XTZUSDT', 'LTCUSDT','DASHUSDT', 'EOSUSDT', 'ETCUSDT', 'ONTUSDT' ]
client = Client(apikey, scrkey, testnet=testnet)
portfolio = Portfolio(client, tradeIns=insIds)
order_size, long, short = portfolio.equity_distribution(longPct=0.5, shortPct=0.5, currency='USDT', orderPct=0.1)
tradeIns = portfolio.position_locks()

print('\nInstruments list : \n', tradeIns, '\n')
print('Order Size : {} '.format(order_size))
print('\nNumber of order : \t#long : {} \t#short : {}  '.format(long, short))

# Problem 3: Request limit

In [ ]:
from datetime import datetime

def klns_to_df(market_data, feats):
    '''
    Add columns' name to dataFrame received from market and select the columns in feats
    
    feats -> ['_o', '_h', '_l', '_c', '_v']
    '''
    fts = list(str(f) for f in feats)
    _df = pd.DataFrame(market_data, columns = ['_t', '_o', '_h', '_l', '_c', '_v', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
    _df[['_o', '_h', '_l', '_c', '_v']] = _df[['_o', '_h', '_l', '_c', '_v']].astype(float)
    return _df[fts]

def candle_no_limit(mkData, interval, startTime, endTime):
    '''
    Return dataFrame of mkData from startTime to endTime
    '''
    min_in_candle ={'1m':1, '3m':3, '5m':5, '10m':10, '15m':15, '30m': 30, \
                    '1h':60, '4h':240, '6h':360, '12h':720, '1d':1440}
    
    if not isinstance(startTime, int): 
        startTime = int(pd.Timestamp(datetime.strptime(startTime, '%d %b %Y %H:%M:%S')).value/10**6)
    if not isinstance(endTime, int): 
        endTime = int(pd.Timestamp(datetime.strptime(endTime, '%d %b %Y %H:%M:%S')).value/10**6)    
    sub_period = int((endTime - startTime)/(min_in_candle[interval]*60*1000))
    sub_start = startTime
    kln_df = None
    while sub_period > 0:
        numklns = min(sub_period, 500)
        market_df = data.candles_data(interval=interval, startTime=sub_start, limit=numklns)
        sub_df = klns_to_df(market_df, ['_t', '_o', '_h', '_l', '_c', '_v'])
        if kln_df is None:
            kln_df = pd.DataFrame(sub_df)
        else:
            kln_df = kln_df.append(pd.DataFrame(sub_df), ignore_index=True)
        sub_period -= numklns
        sub_start += numklns*min_in_candle[interval]*60*1000
    return kln_df

In [ ]:
### Test candle_no_limit() function
symbol = 'BTCUSDT'
data = MarketData(testnet=testnet, symbol=symbol)
t_start = '01 Jun 2020 00:00:00' 
t_end = '03 Jun 2020 00:00:00'
klns = candle_no_limit(data, interval='3m', startTime=t_start, endTime=t_end)
klns.head()